# Reinforcement Learning Street Fighter 2

## Configurar entorno Street Fighter 2

### Instalar y configurar librerias

In [ ]:
# Filtramos los warnings
import warnings
warnings.filterwarnings("ignore")

# Forzar uso CPU en caso de dispones GPU en el PC
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
# instalar librerias
!pip3 install gym gym-retro
#Instalar con pip3, con pip dio problemas al reconocer las roms
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip3 install optuna
!pip3 install stable-baselines3[extra]

In [ ]:
# Import retro para crear el entorno Street Fighter a partir de la ROM
import retro

# Import Time para relentizar el juego
import time

# Comandos a ejecutar en el terminal
# Navegar a la carpeta que contiene/contendrá las roms y descomprimir los .zip con las roms
# Path de la carpeta = ./videoGames_roms/roms
# Ejecutar python -m retro.import . (debe reconocer las roms .md, en caso de que sea para la Sega Genesis,
# si no reconoce se puede ejecutar pip3 install gym gym-retro en la terminal del entorno)

In [ ]:
# verificamos las roms que están importadas
retro.data.list_games()

### Comprobar el entorno SF2

In [ ]:
# Cerramos la variable env por si existiera una instancia
# env.close()

# Instanciamos el entorno, no se pueden intanciar varios entornos por ejecución
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

# Para instanciar varios entornos en paralelo utilizar retrowrapper

In [ ]:
obs = env.reset()
done = False

for game in range(1):
    while not done:
        if(done):
            obs = env.reset()
            
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        time.sleep(0.01)
        print(reward)

#### Métodos útiles

In [ ]:
# env.observation_space() # Verificar entorno
# env.action_space() # Acciones disponibles

## Setup Environment

### Preprocesado del entorno
Con el fin de agilizar el entrenamiento de agente vamos a preprocesar el entorno: comprimiendo los datos del entorno (reduciendo el frame), calculando la variación de los pixel del frame actual con respecto al último para capturar movimiento y aplicarle una escala de grises.

In [ ]:
# Importar librerias
# Import Clase base del entorno para hacer wrapper
from gym import Env

# Import los shapes espaciales para el entorno
from gym.spaces import MultiBinary, Box

# Import numpy para calcular el frame delta
import numpy as np

# Import opencv para aplicar la escala de grises
import cv2

# Import matplotlib para visualizar la imagen
from matplotlib import pyplot as plt

In [ ]:
# Creamos una clase para definir el entorno de SF2
class StreetFighter(Env):
    def __init__(self):
        super().__init__()
        # Especificar el espacio de acciones y el espacio de observaciones
        self.observation_space = Box(low=0, 
                                     high=255, 
                                     shape=(84, 84, 1), 
                                     dtype=np.uint8)
        
        self.action_space = MultiBinary(12)
        
        # Instanciar el entorno
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis',
                               use_restricted_actions=retro.Actions.FILTERED)
    
    def reset(self):
        
        # Devolvemos el primer frame
        obs = self.game.reset()
         
        # Preprocess
        obs = self.preprocess(obs)
        self.previous_frame = obs
        
        # Inicializar atributo para la diferencia de la puntuación
        self.score = 0
        
        return obs
    
    def preprocess(self, observation):
        # Aplicamos el redimensionado del frame y el escalado de grises
        # Escalado de grises
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Redimensionado del frame
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Añadir el valor de los canales
        channel = np.reshape(resize, (84,84,1))
        
        return channel
    
    def step(self, action):
        # Realizar una acción
        obs, reward, done, info = self.game.step(action)
        
        # Procesamos la observación
        obs = self.preprocess(obs)
        
        # Calcular frame delta (Variación en frame anterior y actual)
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs
        
        # Adaptamos la función de recompensa
        reward = info['score'] - self.score
        self.score = info['score']
        
        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs):
        self.game.render()
        
    def close(self):
        self.game.close()


In [ ]:
# Test Clase SF2
#env.close()
env = StreetFighter()

In [ ]:
obs = env.reset()
done = False

for game in range(1):
    while not done:
        if(done):
            obs = env.reset()
            
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        time.sleep(0.01)
        if(reward > 0):
            print(reward)

In [ ]:
env.close()

### Optimización de Hiperparámetros
En este apartado optimizaremos los siguientes hiperparametros del modelo utilizando Optuna:
+ n_step: número máximo de acciones del episodio.
+ gamma: contiene el parémetro que reduce la recompensa por cada acción realizada. 
+ learning_rate: ratio de aprendizaje del algoritmo. 
+ clip_range: 
+ gae_lambda: parámetro de suavización de la ventaja 

In [ ]:
# Importar librerias

# Importar optuna: frame para optimización de parrámetros
import optuna

# Importar algoritmo de entranamiento
from stable_baselines3 import PPO

# Importar métrica de evaluación del modelo
from stable_baselines3.common.evaluation import evaluate_policy

# Importar libreria de Baselines para monitorización
from stable_baselines3.common.monitor import Monitor

# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [ ]:
LOG_DIR = './log/'
OPT_DIR = './opt/'

#### Función para determinar los hiperparámetros a optimizar

In [ ]:
def optimize_ppo(trial):
    return {
        'n_steps': trial.suggest_int('n_steps',2048,8192),
        'gamma': trial.suggest_loguniform('gamma',0.8,0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate',1e-5,1e-4),
        'clip_range': trial.suggest_uniform('clip_range',0.1,0.4),
        'gae_lambda': trial.suggest_uniform('gae_lambda',0.8,0.99)
    }

In [ ]:
def optimize_agent(trial):
    try:
        
        model_params = optimize_ppo(trial)
        
        # Inicializar entorno
        env = StreetFighter()
        env = Monitor(env,LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')
        
        # Definir algoritmo
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        #model.learn(total_timesteps=30000)
        model.learn(total_timesteps=100000)
        
        # Evaluación algoritmo
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()
        
        SAVE_PATH = os.path.join(OPT_DIR,'trial_{}_bestModel'.format(trial.number))
        model.save(SAVE_PATH)
        return mean_reward
        
    except Exception as e:
        print(str(e))
        return -1000

In [ ]:
# Creación del experimento
study = optuna.create_study(direction='maximize')
STUDY_PATH = "./studies/"
#study = optuna.create_study(direction='maximize',storage=STUDY_PATH)
#study.optimize(optimize_agent, n_trials= 10, n_jobs= 1)
study.optimize(optimize_agent, n_trials= 100, n_jobs= 1)

In [ ]:
# Método para obtener los trails del estudio
study.trials

In [ ]:
model = PPO.load(os.path.join(OPT_DIR, 'trial_0_bestModel.zip'))

### Configurar entrenamiento y donde almacenarlo 
En este apartado se definirán los bloques de entrenamiento y donde se almacenará el agente resultante.

In [ ]:
# Importar base callbacks
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

### Entrenar el agente

In [ ]:
# Create environment 
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

model_params = study.best_params

In [ ]:
model_params

In [ ]:
92*64

In [ ]:
# Utilizamos un número de mini-batch que sea multiplo del tamaño del mini-batch para evitar truncar los mini-batch 
# y reducimos el ratio de aprendizaje para evitar el sobreaprendizaje
model_params['n_steps'] = 5888
model_params['learning_rate'] = 8.643050555325267e-10

In [ ]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

# Cargamos el mejor modelo 
model.load(os.path.join(OPT_DIR, 'trial_0_bestModel.zip'))

model.learn(total_timesteps=100000, callback=callback)
#model.learn(total_timesteps=1000000, callback=callback)

### Evaluación del agente

In [ ]:
model = PPO.load('./train/best_model_10000.zip')

In [ ]:
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=1)

### Afinamiento del modelo
En este apartado se va a afinar el agente empleando uno de los agentes resultantes del entrenamiento, para ello se reducirá el ratio de aprendizaje para evitar sobreentrenamiento.

In [ ]:
IMPROVE_PATH = './improveModel/'

In [ ]:
# Reducimos el learning_rate de aprendizaje para disminuir la capacidad de aprendizaje del modelo y evitar sobre entrenamiento
model_params['learning_rate'] = 8.643050555325267e-09

In [ ]:
improve_callback = TrainAndLoggingCallback(check_freq=1000, save_path=IMPROVE_PATH)

In [ ]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

# Cargamos el mejor modelo 
model.load(os.path.join(CHECKPOINT_DIR, 'best_model_20000.zip'))

model.learn(total_timesteps=100000, callback=improve_callback)
#model.learn(total_timesteps=1000000, callback=callback)

In [ ]:
model = PPO.load('./improveModel/best_model_10000.zip')

In [ ]:
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=1)

In [ ]:
mean_reward

### Test Modelo
En este apartado se van a proceder a realizar las pruebas del agente, para ello se realizará una partida y se comprobará el rendimiento del agente.

In [ ]:
obs = env.reset()

In [ ]:
obs.shape

In [ ]:
# Set flag to flase
done = False

In [ ]:
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        #time.sleep(0.01)
        if(reward > 0):
            print(reward)

#### Resultados de las pruebas
Después observar varios partidas del agente, se ha observado que el agente presenta problemas a la hora de calcular distancias, por ejemplo, cuando el oponente está muy cerca del personaje del agente y lanza un ataque de rango, el agente rápidamente lanza el otro para evitar ser golpeado. Por el contrario, cuando el oponente lanza un ataque de rango desde una distancia considerable, el agente responde saltando al momento que el contrincante lanza el ataque lo que provoca que al caer sea golpeado por este ataque, este problema se podría solucionar modificando la diferencia entre los frames (frame_delta) y de esta forma el frame delta podría contener la trayectoria del ataque y no su posición actual solo.

Por otro lado, habría que analizar si el conjunto de acciones del modelo incluye desplazamiento lateral porque cuando este se desplaza lo hace saltando o realizando alguna acción durante el salto, realizar estos tipos de movimientos para desplazarte te deja vulnerable durante la caida.

Por último, no se está puntuando de forma negativa cada vez que el agente recibe un golpe o realiza una acción y no obtiene un resultado positivo, podría ser buena idea puntuarle negativamente este tipo de acciones.

### Notas Trabajo Fin de Máster
Al igual que comenta Nicholas en el video para entrenar este tipo de modelos es necesario realizar un proceso de optimización de los hiperparámetros del orden de cien mil acciones y 100 procesos, y durante el entrnemiento realizar el proceso un millón de veces, a día de hoy no se ha podido hacer pero después de la entrega de Machine Learning avanzado, pretendo realizar este tipo de entrenamiento.

Además creo que debería de guardarse el estudio de optuna para no perderlo y poder reanudar el proceso de ser necesario sin tener que rehacer el estudio de cero.

Por otro lado, me he dado cuenta de que tenía un fallo en el calculo de la recompensa (estaba almacenando en la clase como score la recompensa y no el valor de la puntuación, "self.score = reward") por este motivo se obtenían como resultado puntuaciones tan altas.

He observado que en tensorboard que a partir de los 100.000 aprendizajes el modelo la recompensa decae y no consigue mejorar, quiero realizar un entrenamiento como el que comenta Nicholas y analizar los distintos modelos que genera y como evolucionan en el proceso de entrnamiento.

Me gustaría poder continuar con este proyecto como TFM, me ha parecido muy interesante y divertido verle mejorar, además me parece bastante interesante este tipo de aprendizaje.